# 🛰️ GeoVeil-MP: GNSS Multipath Analysis

**High-Performance RINEX Analysis using Rust with Python Bindings**

## Features
- Multi-GNSS support (GPS, GLONASS, Galileo, BeiDou, QZSS)
- **Optional** SP3 precise orbit integration for accurate elevation angles
- Automatic ephemeris download (can be disabled)
- Interactive visualizations with Plotly
- CSV export for further analysis

## Installation Options
- **Option A**: Install from PyPI (if published) - fastest
- **Option B**: Build from source with Rust - most flexible

---

## 1. 🔧 Installation

### Option A: Install from PyPI (Recommended if available)

In [ ]:
# ============================================================
# OPTION A: Install pre-built wheel from PyPI or local wheel
# ============================================================
# This is the fastest method if a wheel is available

import subprocess
import sys
import os
import glob

print("📦 Installing geoveil-mp...")
print("=" * 50)

# Upgrade pip first
subprocess.check_call([sys.executable, '-m', 'pip', 'install', '-q', '--upgrade', 'pip'])

# Try to install from PyPI (if published)
try:
    result = subprocess.run(
        [sys.executable, '-m', 'pip', 'install', '--upgrade', 'geoveil-mp'],
        capture_output=True, text=True, timeout=120
    )
    if result.returncode == 0:
        print("✅ Installed from PyPI!")
    else:
        raise Exception("Not on PyPI")
except Exception as e:
    print(f"ℹ️ PyPI install failed: {e}")
    
    # Try local wheel files
    wheels = glob.glob(os.path.join(os.getcwd(), '*.whl'))
    wheels += glob.glob(os.path.join(os.getcwd(), 'dist', '*.whl'))
    wheels += glob.glob(os.path.join(os.getcwd(), 'geoveil-mp', 'target', 'wheels', '*.whl'))
    
    if wheels:
        wheel = max(wheels, key=os.path.getctime)  # Most recent
        print(f"\n📦 Found local wheel: {os.path.basename(wheel)}")
        result = subprocess.run(
            [sys.executable, '-m', 'pip', 'install', '--upgrade', '--force-reinstall', wheel],
            capture_output=True, text=True
        )
        if result.returncode == 0:
            print("✅ Installed from local wheel!")
        else:
            print(f"❌ Wheel install failed: {result.stderr}")
            print("\n👉 Try Option B: Build from source (next cell)")
    else:
        print("\n⚠️ No wheel found. Run Option B to build from source.")

# Install other dependencies
print("\n📦 Installing analysis dependencies...")
deps = ['numpy', 'pandas', 'matplotlib', 'plotly', 'ipywidgets', 'tqdm']
for dep in deps:
    try:
        __import__(dep.replace('-', '_'))
    except ImportError:
        print(f"   Installing {dep}...")
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', '-q', dep])

print("\n✅ Dependencies installed!")

# Test import
try:
    import geoveil_mp as gm
    print(f"\n🎉 geoveil_mp v{gm.version()} loaded successfully!")
except ImportError:
    print("\n⚠️ geoveil_mp not loaded. Run Option B below to build from source.")

### Option B: Build from Source (Requires Rust)

In [ ]:
# ============================================================
# OPTION B: Build from Rust source code
# ============================================================
# Requires: Rust (https://rustup.rs)
# Skip this if Option A worked!

import subprocess
import sys
import os
import platform
import glob

# Check if already installed
try:
    import geoveil_mp as gm
    print(f"✅ geoveil_mp v{gm.version()} already installed!")
    print("   Skip this cell - library is ready.")
except ImportError:
    print("📦 Building geoveil-mp from source...")
    print("=" * 60)
    
    # Install maturin
    subprocess.check_call([sys.executable, '-m', 'pip', 'install', '-q', 'maturin>=1.4'])
    
    # Find library path
    RUST_LIB_PATH = None
    possible_paths = [
        os.path.join(os.getcwd(), 'geoveil-mp'),
        os.path.join(os.getcwd(), 'geoveil_mp'),
        os.path.join(os.path.dirname(os.getcwd()), 'geoveil-mp'),
    ]
    for p in possible_paths:
        if os.path.exists(os.path.join(p, 'Cargo.toml')):
            RUST_LIB_PATH = p
            break
    
    if not RUST_LIB_PATH:
        print("❌ Rust library not found!")
        print("   Please extract geoveil-mp source to the notebook directory")
        raise FileNotFoundError("geoveil-mp directory not found")
    
    print(f"📁 Source path: {RUST_LIB_PATH}")
    print(f"🖥️ Platform: {platform.system()} {platform.machine()}")
    
    # Add cargo to PATH
    cargo_bin = os.path.expanduser("~/.cargo/bin")
    if os.path.exists(cargo_bin):
        os.environ['PATH'] = f"{cargo_bin}{os.pathsep}{os.environ.get('PATH', '')}"
    
    # Check Rust
    print("\n🔧 Checking Rust...")
    result = subprocess.run(['cargo', '--version'], capture_output=True, text=True,
                           shell=(platform.system() == 'Windows'))
    if result.returncode != 0:
        print("❌ Cargo not found!")
        print("   Install Rust from: https://rustup.rs")
        print("   Then restart your kernel")
        raise RuntimeError("Rust not installed")
    print(f"✅ {result.stdout.strip()}")
    
    # Build
    print("\n🔨 Building (this may take 1-3 minutes)...")
    print("-" * 60)
    
    env = os.environ.copy()
    env['PYTHONIOENCODING'] = 'utf-8'
    
    if platform.system() == 'Windows':
        build_cmd = f'cd /d "{RUST_LIB_PATH}" && "{sys.executable}" -m maturin develop --release --features python'
        ret = os.system(build_cmd)
        if ret != 0:
            raise RuntimeError(f"Build failed with code {ret}")
    else:
        # Try maturin develop first
        result = subprocess.run(
            [sys.executable, '-m', 'maturin', 'develop', '--release', '--features', 'python'],
            cwd=RUST_LIB_PATH, env=env, capture_output=True, text=True
        )
        
        if result.returncode != 0:
            print("⚠️ maturin develop failed, trying build + pip install...")
            # Build wheel
            subprocess.run(
                [sys.executable, '-m', 'maturin', 'build', '--release', '--features', 'python'],
                cwd=RUST_LIB_PATH, env=env, check=True
            )
            # Install wheel
            wheels = glob.glob(os.path.join(RUST_LIB_PATH, 'target/wheels/*.whl'))
            if wheels:
                wheel = max(wheels, key=os.path.getctime)
                subprocess.run(
                    [sys.executable, '-m', 'pip', 'install', '--force-reinstall', '--no-deps', wheel],
                    check=True
                )
        else:
            print(result.stdout)
    
    print("-" * 60)
    print("✅ Build complete!")
    
    # Test import
    if 'geoveil_mp' in sys.modules:
        del sys.modules['geoveil_mp']
    import geoveil_mp as gm
    print(f"\n🎉 geoveil_mp v{gm.version()} loaded!")

In [ ]:
# Quick library check
try:
    import geoveil_mp as gm
    print(f"✅ geoveil_mp v{gm.version()} ready!")
    print(f"   Speed of light: {gm.SPEED_OF_LIGHT:,.0f} m/s")
    print(f"   GM_WGS84: {gm.GM_WGS84:.6e} m³/s²")
except ImportError as e:
    print(f"❌ Library not loaded: {e}")
    print("   Run Option A or Option B above")
    gm = None

## 2. 📚 Library API Overview

In [ ]:
if gm:
    print("📖 geoveil_mp API Reference")
    print("=" * 50)
    print("\nClasses:")
    print("  • GnssSystem, Satellite, Epoch, Ecef, Geodetic")
    print("  • RinexObsData, Sp3Data, MultipathAnalyzer, AnalysisResults")
    print("\nFunctions:")
    print("  • read_rinex_obs(path), read_rinex_obs_bytes(data, filename)")
    print("  • read_sp3(path) - OPTIONAL for precise elevations")
    print("  • compute_elevation(sp3, receiver, sat, epoch)")
    print("  • get_frequency(system, band, fcn), get_wavelength(...)")
    print("\nConstants:")
    print(f"  • SPEED_OF_LIGHT, GM_WGS84, EARTH_RADIUS")

## 3. 🧪 Basic Examples

In [ ]:
if gm:
    # Epoch example
    epoch = gm.Epoch(2024, 6, 15, 12, 30, 45.5)
    print(f"Epoch: {epoch}")
    print(f"  GPS Time: Week {epoch.to_gps_time()[0]}, TOW {epoch.to_gps_time()[1]:.3f}s")
    print(f"  Julian Date: {epoch.julian_date():.6f}")
    
    # Coordinate example
    ecef = gm.Ecef(4000000.0, 1000000.0, 4800000.0)
    geo = ecef.to_geodetic()
    print(f"\nCoordinates: {geo.lat:.4f}°N, {geo.lon:.4f}°E, {geo.height:.0f}m")
    
    # Frequencies
    print(f"\nGPS L1 frequency: {gm.get_frequency('G', 1)/1e6:.3f} MHz")

## 4. 📤 File Upload and Analysis Setup

In [ ]:
import ipywidgets as widgets
from IPython.display import display, HTML, clear_output
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
import gzip
import io
import os
import glob
import urllib.request
import datetime
import ssl

# Storage
uploaded_files = {
    'obs_content': None,
    'obs_filename': None,
    'sp3_path': None,
}
results_df = None
stats_df = None

NOTEBOOK_DIR = os.getcwd()
print(f"📁 Current directory: {NOTEBOOK_DIR}")

# List available files
print("\n📋 Available RINEX files in directory:")
found_obs = []
for pattern in ['*.rnx', '*.RNX', '*.obs', '*.OBS', '*.??o', '*.??O', '*.gz']:
    found_obs.extend(glob.glob(os.path.join(NOTEBOOK_DIR, pattern)))
if found_obs:
    for f in sorted(set(found_obs))[:10]:
        print(f"  - {os.path.basename(f)} ({os.path.getsize(f)/(1024*1024):.1f} MB)")
else:
    print("  No observation files found")

# List SP3 files
print("\n📋 Available SP3/ephemeris files:")
found_sp3 = []
for pattern in ['*.sp3', '*.SP3', '*.eph', '*.EPH']:
    found_sp3.extend(glob.glob(os.path.join(NOTEBOOK_DIR, pattern)))
if found_sp3:
    for f in sorted(set(found_sp3))[:5]:
        print(f"  - {os.path.basename(f)}")
else:
    print("  No SP3 files found (can auto-download or skip)")

print("\n✅ Storage initialized")

In [ ]:
# SP3 Downloader Functions

def gps_week_from_date(year, month, day):
    """Calculate GPS week from date."""
    gps_epoch = datetime.date(1980, 1, 6)
    target = datetime.date(year, month, day)
    delta = (target - gps_epoch).days
    return delta // 7, delta % 7

def decompress_file(filepath):
    """Decompress .gz file."""
    if filepath.endswith('.gz'):
        try:
            output_path = filepath[:-3]
            with gzip.open(filepath, 'rb') as f_in:
                with open(output_path, 'wb') as f_out:
                    f_out.write(f_in.read())
            return output_path
        except Exception as e:
            print(f"      gzip error: {e}")
            return None
    return filepath

def download_sp3(year, month, day, output_dir=None):
    """
    Download SP3 precise orbit file for a given date.
    Sources: ESA, GFZ, CODE, WHU (no auth required)
    """
    if output_dir is None:
        output_dir = NOTEBOOK_DIR
    
    week, dow = gps_week_from_date(year, month, day)
    doy = datetime.date(year, month, day).timetuple().tm_yday
    
    print(f"📡 Downloading SP3 for {year}-{month:02d}-{day:02d}")
    print(f"   GPS Week: {week}, DOY: {doy:03d}")
    
    ctx = ssl.create_default_context()
    ctx.check_hostname = False
    ctx.verify_mode = ssl.CERT_NONE
    
    sources = [
        {"name": "ESA Multi-GNSS Final", "systems": "G,R,E,C,J",
         "url": f"http://navigation-office.esa.int/products/gnss-products/{week}/ESA0MGNFIN_{year}{doy:03d}0000_01D_05M_ORB.SP3.gz",
         "filename": f"ESA0MGNFIN_{year}{doy:03d}0000_01D_05M_ORB.SP3.gz"},
        {"name": "ESA Rapid", "systems": "G,R,E,C,J",
         "url": f"http://navigation-office.esa.int/products/gnss-products/{week}/ESA0MGNRAP_{year}{doy:03d}0000_01D_05M_ORB.SP3.gz",
         "filename": f"ESA0MGNRAP_{year}{doy:03d}0000_01D_05M_ORB.SP3.gz"},
        {"name": "GFZ Multi-GNSS (BKG)", "systems": "G,R,E,C,J",
         "url": f"https://igs.bkg.bund.de/root_ftp/IGS/products/mgex/{week}/GFZ0MGXFIN_{year}{doy:03d}0000_01D_05M_ORB.SP3.gz",
         "filename": f"GFZ0MGXFIN_{year}{doy:03d}0000_01D_05M_ORB.SP3.gz"},
        {"name": "GFZ Rapid (BKG)", "systems": "G,R,E,C,J",
         "url": f"https://igs.bkg.bund.de/root_ftp/IGS/products/mgex/{week}/GFZ0MGXRAP_{year}{doy:03d}0000_01D_05M_ORB.SP3.gz",
         "filename": f"GFZ0MGXRAP_{year}{doy:03d}0000_01D_05M_ORB.SP3.gz"},
        {"name": "CODE Multi-GNSS (BKG)", "systems": "G,R,E,C",
         "url": f"https://igs.bkg.bund.de/root_ftp/IGS/products/mgex/{week}/COD0MGXFIN_{year}{doy:03d}0000_01D_05M_ORB.SP3.gz",
         "filename": f"COD0MGXFIN_{year}{doy:03d}0000_01D_05M_ORB.SP3.gz"},
        {"name": "WHU Multi-GNSS (BKG)", "systems": "G,R,E,C,J",
         "url": f"https://igs.bkg.bund.de/root_ftp/IGS/products/mgex/{week}/WUM0MGXFIN_{year}{doy:03d}0000_01D_05M_ORB.SP3.gz",
         "filename": f"WUM0MGXFIN_{year}{doy:03d}0000_01D_05M_ORB.SP3.gz"},
        {"name": "IGS GPS-only (BKG)", "systems": "G",
         "url": f"https://igs.bkg.bund.de/root_ftp/IGS/products/{week}/IGS0OPSFIN_{year}{doy:03d}0000_01D_15M_ORB.SP3.gz",
         "filename": f"IGS0OPSFIN_{year}{doy:03d}0000_01D_15M_ORB.SP3.gz"},
    ]
    
    for source in sources:
        output_path = os.path.join(output_dir, source["filename"])
        print(f"   [{source['systems']}] {source['name']}...", end=" ")
        
        try:
            req = urllib.request.Request(source["url"], headers={'User-Agent': 'GNSS-Tool/1.0'})
            with urllib.request.urlopen(req, timeout=60, context=ctx) as response:
                data = response.read()
            
            if len(data) < 10000:
                print(f"too small")
                continue
            
            with open(output_path, 'wb') as f:
                f.write(data)
            
            decompressed = decompress_file(output_path)
            if decompressed:
                with open(decompressed, 'r') as f:
                    if f.readline().startswith('#'):
                        print(f"✅ {len(data)//1024}KB")
                        return decompressed
            print("invalid")
        except urllib.error.HTTPError as e:
            print(f"HTTP {e.code}")
        except Exception as e:
            print(f"{type(e).__name__}")
    
    print("\n⚠️ SP3 download failed (Final products have ~14 day delay)")
    return None

print("✅ SP3 downloader ready")

In [ ]:
# File Input Widgets

obs_upload = widgets.FileUpload(
    accept='.obs,.rnx,.crx,.??o,.gz,*',
    multiple=False, description='Observation', button_style='info'
)

obs_path_input = widgets.Text(
    value='', placeholder='filename.rnx or /full/path/to/file.rnx',
    description='OBS File:', style={'description_width': 'initial'},
    layout=widgets.Layout(width='500px')
)

load_btn = widgets.Button(description='📥 Load RINEX', button_style='warning',
                         layout=widgets.Layout(width='150px'))

# ============================================================
# EPHEMERIS OPTIONS - NOW FULLY OPTIONAL!
# ============================================================
sp3_options_header = widgets.HTML(value="<b>📡 Ephemeris Options (Optional):</b>")

# Master switch - enable/disable ephemeris entirely
sp3_enabled = widgets.Checkbox(
    value=True, 
    description='Use ephemeris for accurate elevations',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='350px')
)

# Auto-download option (only active if ephemeris enabled)
sp3_auto_download = widgets.Checkbox(
    value=True, 
    description='Auto-download SP3 based on RINEX date',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='350px')
)

# Manual path (for user's own ephemeris)
sp3_path_input = widgets.Text(
    value='', 
    placeholder='(Optional) Path to your own SP3/ephemeris file',
    description='SP3 Path:', 
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='550px')
)

sp3_status = widgets.HTML(value="")

# Link checkboxes - disable auto-download if ephemeris is disabled
def on_sp3_enabled_change(change):
    if not change['new']:
        sp3_auto_download.value = False
        sp3_auto_download.disabled = True
        sp3_path_input.disabled = True
        sp3_status.value = "<small>ℹ️ Elevations will use approximate values</small>"
    else:
        sp3_auto_download.disabled = False
        sp3_path_input.disabled = False
        sp3_status.value = ""

sp3_enabled.observe(on_sp3_enabled_change, names='value')

# Analysis settings
elevation_slider = widgets.FloatSlider(
    value=5.0, min=0.0, max=30.0, step=1.0,
    description='Elevation Cutoff (°):', style={'description_width': 'initial'},
    layout=widgets.Layout(width='400px')
)

system_checks = {
    'G': widgets.Checkbox(value=True, description='GPS'),
    'R': widgets.Checkbox(value=True, description='GLONASS'),
    'E': widgets.Checkbox(value=True, description='Galileo'),
    'C': widgets.Checkbox(value=False, description='BeiDou'),
}

analyze_btn = widgets.Button(description='🔬 Run Analysis', button_style='primary',
                            layout=widgets.Layout(width='200px', height='40px'))
progress = widgets.FloatProgress(value=0, min=0, max=1.0, description='Progress:',
                                 layout=widgets.Layout(width='400px', visibility='hidden'))
status = widgets.HTML(value="<b>Status:</b> Ready")
output = widgets.Output()

def resolve_path(path_or_filename):
    if not path_or_filename:
        return None
    path_or_filename = path_or_filename.strip()
    if os.path.isabs(path_or_filename) or os.path.dirname(path_or_filename):
        return path_or_filename
    return os.path.join(NOTEBOOK_DIR, path_or_filename)

def load_files(btn):
    """Load RINEX and optionally download/load SP3."""
    with output:
        clear_output()
        print("📥 Loading files...")
        
        obs_loaded = False
        
        # Try file path first
        if obs_path_input.value.strip():
            path = resolve_path(obs_path_input.value)
            print(f"\n📂 Loading: {path}")
            
            if os.path.exists(path):
                try:
                    with open(path, 'rb') as f:
                        content = f.read()
                    name = os.path.basename(path)
                    if name.lower().endswith('.gz'):
                        print("   Decompressing...")
                        content = gzip.decompress(content)
                    uploaded_files['obs_content'] = content
                    uploaded_files['obs_filename'] = name
                    print(f"✅ Loaded: {name} ({len(content)/(1024*1024):.1f} MB)")
                    obs_loaded = True
                except Exception as e:
                    print(f"❌ Error: {e}")
            else:
                print(f"❌ File not found: {path}")
        
        # Try upload widget
        if not obs_loaded and obs_upload.value:
            try:
                if len(obs_upload.value) > 0:
                    if isinstance(obs_upload.value, dict):
                        name = list(obs_upload.value.keys())[0]
                        content = obs_upload.value[name]['content']
                    else:
                        file_info = obs_upload.value[0]
                        name = getattr(file_info, 'name', 'unknown')
                        content = getattr(file_info, 'content', b'')
                    
                    if content:
                        if name.lower().endswith('.gz'):
                            content = gzip.decompress(content)
                        uploaded_files['obs_content'] = content
                        uploaded_files['obs_filename'] = name
                        print(f"✅ Uploaded: {name} ({len(content)/(1024*1024):.1f} MB)")
                        obs_loaded = True
            except Exception as e:
                print(f"❌ Upload error: {e}")
        
        if not obs_loaded:
            print("\n⚠️ No observation file loaded!")
            status.value = "<b>Status:</b> ⚠️ No file loaded"
            return
        
        # ============================================================
        # EPHEMERIS HANDLING - NOW OPTIONAL!
        # ============================================================
        uploaded_files['sp3_path'] = None  # Reset
        
        if not sp3_enabled.value:
            # User chose to skip ephemeris
            print("\nℹ️ Ephemeris disabled - using approximate elevations")
            sp3_status.value = "<b style='color:gray'>SP3:</b> Disabled"
        elif sp3_path_input.value.strip():
            # User provided their own SP3 file
            path = resolve_path(sp3_path_input.value)
            if os.path.exists(path):
                uploaded_files['sp3_path'] = path
                sp3_status.value = f"<b style='color:green'>SP3:</b> ✅ {os.path.basename(path)}"
                print(f"\n✅ SP3: {os.path.basename(path)}")
            else:
                print(f"\n⚠️ SP3 file not found: {path}")
                sp3_status.value = f"<b style='color:orange'>SP3:</b> ⚠️ File not found"
        elif sp3_auto_download.value:
            # Auto-download based on RINEX date
            print("\n" + "="*50)
            print("📅 Reading date from RINEX for SP3 download...")
            try:
                content = uploaded_files['obs_content']
                if isinstance(content, memoryview):
                    content = bytes(content)
                
                obs_data = gm.read_rinex_obs_bytes(content, uploaded_files['obs_filename'])
                first_epoch = obs_data.first_epoch()
                
                if first_epoch:
                    year, month, day = first_epoch.year, first_epoch.month, first_epoch.day
                    print(f"✅ RINEX date: {year}-{month:02d}-{day:02d}")
                    print("")
                    
                    sp3_path = download_sp3(year, month, day)
                    if sp3_path:
                        uploaded_files['sp3_path'] = sp3_path
                        sp3_path_input.value = sp3_path
                        sp3_status.value = f"<b style='color:green'>SP3:</b> ✅ {os.path.basename(sp3_path)}"
                    else:
                        sp3_status.value = "<b style='color:orange'>SP3:</b> ⚠️ Download failed (using approximate elevations)"
                else:
                    print("❌ Could not read date from RINEX")
            except Exception as e:
                print(f"❌ Error reading RINEX: {e}")
                sp3_status.value = "<b style='color:red'>SP3:</b> ❌ Error"
        else:
            # Ephemeris enabled but no auto-download and no manual path
            print("\nℹ️ No SP3 specified - elevations will be approximate")
            sp3_status.value = "<b style='color:gray'>SP3:</b> None (approximate elevations)"
        
        # Update status
        obs_size = len(uploaded_files['obs_content']) / (1024*1024)
        status.value = f"<b>Status:</b> ✅ Ready - {uploaded_files['obs_filename']} ({obs_size:.1f} MB)"

load_btn.on_click(load_files)
print("✅ File input ready")

In [ ]:
def run_analysis(btn):
    """Run multipath analysis."""
    global results_df, stats_df
    
    import datetime as dt
    import time
    
    timestamp = dt.datetime.now().strftime("%Y%m%d_%H%M%S")
    
    with output:
        clear_output()
        
        content = uploaded_files.get('obs_content')
        if content is None:
            print("❌ No observation file loaded")
            status.value = "<b>Status:</b> ❌ No file loaded"
            return
        
        print("=" * 60)
        print("🔬 GNSS MULTIPATH ANALYSIS")
        print("=" * 60)
        
        progress.layout.visibility = 'visible'
        progress.value = 0.1
        
        try:
            # Parse RINEX
            print("\n📄 Parsing RINEX...")
            if isinstance(content, memoryview):
                content = bytes(content)
            
            start = time.time()
            obs_data = gm.read_rinex_obs_bytes(content, uploaded_files.get('obs_filename', 'obs.rnx'))
            print(f"   ✅ Parsed in {time.time()-start:.2f}s")
            print(f"   Epochs: {obs_data.num_epochs}, Satellites: {obs_data.num_satellites}")
            print(f"   First: {obs_data.first_epoch()}, Last: {obs_data.last_epoch()}")
            
            pos = obs_data.approx_position
            if pos:
                geo = pos.to_geodetic()
                print(f"   Position: {geo.lat:.4f}°N, {geo.lon:.4f}°E, {geo.height:.0f}m")
            
            progress.value = 0.3
            
            # Load SP3 (if enabled and available)
            sp3_data = None
            if sp3_enabled.value:
                sp3_path = uploaded_files.get('sp3_path') or sp3_path_input.value.strip()
                if sp3_path:
                    sp3_path = resolve_path(sp3_path)
                    if sp3_path and os.path.exists(sp3_path):
                        print(f"\n📡 Loading SP3: {os.path.basename(sp3_path)}")
                        try:
                            sp3_data = gm.read_sp3(sp3_path)
                            print(f"   ✅ {sp3_data.num_satellites} satellites, {sp3_data.num_epochs} epochs")
                        except Exception as e:
                            print(f"   ⚠️ SP3 load error: {e}")
                            sp3_data = None
            
            if sp3_data is None:
                print("\nℹ️ No SP3 - elevations will be placeholder values")
            
            progress.value = 0.4
            
            # Run analysis
            print("\n🔬 Computing multipath...")
            selected_systems = [sys for sys, cb in system_checks.items() if cb.value]
            print(f"   Systems: {selected_systems}, Cutoff: {elevation_slider.value}°")
            
            start = time.time()
            analyzer = gm.MultipathAnalyzer(obs_data, elevation_cutoff=elevation_slider.value, systems=selected_systems)
            results = analyzer.analyze()
            print(f"   ✅ {results.total_estimates()} estimates in {time.time()-start:.2f}s")
            
            progress.value = 0.6
            
            # Compute elevations (only if SP3 available)
            if sp3_data and pos:
                print("\n📐 Computing precise elevations from SP3...")
                start = time.time()
                try:
                    computed, failed = results.compute_elevations(sp3_data, pos)
                    print(f"   ✅ {computed} computed, {failed} failed ({time.time()-start:.2f}s)")
                except Exception as e:
                    print(f"   ⚠️ Elevation computation error: {e}")
            
            progress.value = 0.8
            
            # Create DataFrames
            print("\n📊 Creating DataFrames...")
            results_data = [{'satellite': e.satellite, 'system': e.system, 'epoch': e.epoch,
                            'mp_value': e.mp_value, 'elevation': e.elevation, 'azimuth': e.azimuth,
                            'snr': e.snr, 'signal': e.signal} for e in results.estimates]
            results_df = pd.DataFrame(results_data)
            
            stats_data = [{'signal': s.signal, 'count': s.count, 'rms': s.rms, 'mean': s.mean,
                          'std_dev': s.std_dev, 'min': s.min, 'max': s.max} for s in results.statistics]
            stats_df = pd.DataFrame(stats_data)
            
            # Print statistics
            print("\n" + "=" * 60)
            print("MULTIPATH STATISTICS")
            print("=" * 60)
            print(f"\n{'Signal':<12} {'Count':>8} {'RMS (m)':>10} {'Mean (m)':>10}")
            print("-" * 42)
            for _, row in stats_df.iterrows():
                print(f"{row['signal']:<12} {row['count']:>8} {row['rms']:>10.4f} {row['mean']:>10.4f}")
            
            # Save CSVs
            mp_file = f'multipath_data_{timestamp}.csv'
            stats_file = f'multipath_stats_{timestamp}.csv'
            results_df.to_csv(mp_file, index=False)
            stats_df.to_csv(stats_file, index=False)
            print(f"\n✅ Saved: {mp_file}, {stats_file}")
            
            progress.value = 1.0
            status.value = f"<b>Status:</b> ✅ Complete! {len(results_df)} estimates"
            
            from IPython.display import FileLink
            display(FileLink(mp_file))
            display(FileLink(stats_file))
            
        except Exception as e:
            print(f"\n❌ Error: {e}")
            import traceback
            traceback.print_exc()
            status.value = f"<b>Status:</b> ❌ Error: {e}"

analyze_btn.on_click(run_analysis)
print("✅ Analysis function ready")

## 5. 📤 Upload Files and Run Analysis

In [ ]:
# Display upload interface
display(HTML("<h3>🛰️ Load RINEX Files</h3>"))

display(HTML("<p><b>Option A - Upload (small files &lt;50MB):</b></p>"))
display(obs_upload)

display(HTML("<p><b>Option B - File Path (large files, recommended):</b></p>"))
display(obs_path_input)

display(HTML("<br>"))
display(load_btn)

display(HTML("<hr>"))
display(sp3_options_header)
display(HTML("<small>Ephemeris provides accurate satellite elevation angles. Can be disabled for quick analysis.</small>"))
display(sp3_enabled)
display(sp3_auto_download)
display(sp3_path_input)
display(sp3_status)

display(HTML("<hr><h4>⚙️ Analysis Settings</h4>"))
display(elevation_slider)
display(HTML("<b>GNSS Systems:</b>"))
display(widgets.HBox(list(system_checks.values())))

display(HTML("<br>"))
display(widgets.HBox([analyze_btn, progress]))
display(status)
display(HTML("<hr>"))
display(output)

## 6. 📈 Visualization

In [ ]:
# System colors and names
SYSTEM_COLORS = {
    'G': '#1f77b4',  # GPS - blue
    'R': '#ff7f0e',  # GLONASS - orange
    'E': '#2ca02c',  # Galileo - green
    'C': '#d62728',  # BeiDou - red
    'J': '#9467bd',  # QZSS - purple
    'I': '#8c564b',  # NavIC - brown
}
SYSTEM_NAMES = {
    'G': 'GPS', 'R': 'GLONASS', 'E': 'Galileo', 
    'C': 'BeiDou', 'J': 'QZSS', 'I': 'NavIC'
}

def get_df():
    global results_df
    try:
        if results_df is not None and len(results_df) > 0:
            return results_df
    except:
        pass
    return None

print("✅ Visualization helpers ready")

In [ ]:
# Multipath Time Series
df = get_df()
if df is not None:
    df_plot = df.copy()
    try:
        df_plot['epoch_dt'] = pd.to_datetime(df_plot['epoch'])
        x_col = 'epoch_dt'
    except:
        x_col = 'epoch'
    
    fig = px.scatter(df_plot, x=x_col, y='mp_value', color='system',
                    color_discrete_map=SYSTEM_COLORS,
                    title='Multipath vs Time',
                    labels={'mp_value': 'Multipath (m)', x_col: 'Time'},
                    hover_data=['satellite', 'signal', 'elevation'])
    fig.add_hline(y=0, line_dash='dash', line_color='gray', opacity=0.5)
    fig.update_layout(height=500)
    fig.show()
else:
    print("⚠️ Run analysis first")

In [ ]:
# RMS Bar Plot
try:
    if stats_df is not None and len(stats_df) > 0:
        sdf = stats_df.sort_values('rms', ascending=True)
        fig = go.Figure(go.Bar(
            x=sdf['signal'], y=sdf['rms'],
            text=[f"{v:.4f}" for v in sdf['rms']],
            textposition='outside',
            marker_color='steelblue'
        ))
        fig.update_layout(title='Multipath RMS by Signal', xaxis_title='Signal',
                         yaxis_title='RMS (m)', height=500)
        fig.show()
    else:
        print("⚠️ Run analysis first")
except NameError:
    print("⚠️ Run analysis first")

In [ ]:
# Multipath vs Elevation
df = get_df()
if df is not None:
    fig = px.scatter(df, x='elevation', y='mp_value', color='system',
                    color_discrete_map=SYSTEM_COLORS,
                    title='Multipath vs Elevation',
                    labels={'mp_value': 'Multipath (m)', 'elevation': 'Elevation (°)'},
                    hover_data=['satellite', 'signal'])
    fig.add_hline(y=0, line_dash='dash', line_color='gray', opacity=0.5)
    fig.update_layout(height=500)
    fig.show()
else:
    print("⚠️ Run analysis first")

In [ ]:
# Skyplot
df = get_df()
if df is not None and df['azimuth'].abs().sum() > 0:
    df_plot = df.copy()
    df_plot['r'] = 90 - df_plot['elevation']
    
    fig = go.Figure(go.Scatterpolar(
        r=df_plot['r'], theta=df_plot['azimuth'], mode='markers',
        marker=dict(size=4, color=df_plot['mp_value'], colorscale='RdBu_r',
                   cmin=-1.5, cmax=1.5, colorbar=dict(title='MP (m)')),
        text=df_plot['satellite'],
        hovertemplate='%{text}<br>Az: %{theta}°<br>El: %{customdata}°<br>MP: %{marker.color:.3f}m',
        customdata=df_plot['elevation']
    ))
    fig.update_layout(title='Multipath Skyplot',
                     polar=dict(radialaxis=dict(range=[0, 90], tickvals=[30, 60, 90]),
                               angularaxis=dict(direction='clockwise', rotation=90)),
                     height=600, width=600)
    fig.show()
else:
    print("⚠️ Run analysis first or no azimuth data")

In [ ]:
# Per-satellite statistics
df = get_df()
if df is not None:
    sat_stats = df.groupby('satellite').agg({
        'mp_value': ['count', lambda x: np.sqrt(np.mean(x**2)), 'mean', 'std']
    }).round(4)
    sat_stats.columns = ['Count', 'RMS (m)', 'Mean (m)', 'Std (m)']
    sat_stats = sat_stats.sort_values('RMS (m)', ascending=False)
    print("📊 Per-Satellite Statistics (Top 20)")
    display(sat_stats.head(20))
else:
    print("⚠️ Run analysis first")

## 7. 💾 Export Results

In [ ]:
def export_results():
    """Export results with plots."""
    global results_df, stats_df
    
    try:
        if results_df is None or results_df.empty:
            print("⚠️ No results. Run analysis first.")
            return
    except:
        print("⚠️ No results. Run analysis first.")
        return
    
    from datetime import datetime
    timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
    output_dir = f"export_{timestamp}"
    os.makedirs(output_dir, exist_ok=True)
    
    print(f"📁 Exporting to: {output_dir}/")
    
    # CSVs
    results_df.to_csv(os.path.join(output_dir, 'multipath_data.csv'), index=False)
    if stats_df is not None:
        stats_df.to_csv(os.path.join(output_dir, 'multipath_stats.csv'), index=False)
    print("✅ CSVs saved")
    
    # Plots
    import matplotlib
    matplotlib.use('Agg')
    import matplotlib.pyplot as plt
    
    # RMS by signal
    if stats_df is not None:
        fig, ax = plt.subplots(figsize=(12, 6))
        colors = [SYSTEM_COLORS.get(s[0], '#333') for s in stats_df['signal']]
        ax.bar(stats_df['signal'], stats_df['rms'], color=colors, alpha=0.8)
        ax.set_xlabel('Signal'); ax.set_ylabel('RMS (m)')
        ax.set_title('Multipath RMS by Signal')
        plt.tight_layout()
        plt.savefig(os.path.join(output_dir, 'rms_by_signal.png'), dpi=150)
        plt.close()
    
    # MP vs Elevation
    df = results_df[results_df['elevation'] > 0]
    if len(df) > 0:
        fig, ax = plt.subplots(figsize=(12, 6))
        for sys in df['system'].unique():
            sdf = df[df['system'] == sys]
            ax.scatter(sdf['elevation'], sdf['mp_value'], s=1, alpha=0.3,
                      color=SYSTEM_COLORS.get(sys, '#333'), label=SYSTEM_NAMES.get(sys, sys))
        ax.set_xlabel('Elevation (°)'); ax.set_ylabel('Multipath (m)')
        ax.set_title('Multipath vs Elevation'); ax.legend(markerscale=5)
        ax.set_xlim(0, 90)
        plt.tight_layout()
        plt.savefig(os.path.join(output_dir, 'mp_vs_elevation.png'), dpi=150)
        plt.close()
    
    # Skyplot
    df = results_df[(results_df['elevation'] > 0) & (results_df['azimuth'] != 0)]
    if len(df) > 0:
        fig, ax = plt.subplots(figsize=(10, 10), subplot_kw={'projection': 'polar'})
        theta = np.radians(df['azimuth'])
        r = 90 - df['elevation']
        scatter = ax.scatter(theta, r, c=np.clip(df['mp_value'], -1.5, 1.5),
                           cmap='RdBu_r', s=2, alpha=0.5, vmin=-1.5, vmax=1.5)
        ax.set_theta_zero_location('N'); ax.set_theta_direction(-1)
        ax.set_ylim(0, 90); ax.set_title('Multipath Skyplot')
        plt.colorbar(scatter, ax=ax, shrink=0.8, label='MP (m)')
        plt.tight_layout()
        plt.savefig(os.path.join(output_dir, 'skyplot.png'), dpi=150)
        plt.close()
    
    print("✅ Plots saved")
    
    from IPython.display import FileLink, HTML
    display(HTML("<b>📁 Exported:</b>"))
    for f in os.listdir(output_dir):
        display(FileLink(os.path.join(output_dir, f)))

export_btn = widgets.Button(description='📊 Export All', button_style='success')
export_output = widgets.Output()

def on_export(btn):
    with export_output:
        clear_output()
        export_results()

export_btn.on_click(on_export)
display(export_btn)
display(export_output)

## 📋 Reference

### Multipath Linear Combination

$$MP_1 = R_1 - \left(1 + \frac{2}{\alpha - 1}\right)\Phi_1 + \frac{2}{\alpha - 1}\Phi_2$$

### Quality Thresholds

| Parameter | Good | Acceptable | Poor |
|-----------|------|------------|------|
| MP RMS | < 0.3m | 0.3-0.5m | > 0.5m |

In [ ]:
print("🛰️ GeoVeil-MP Analysis Notebook v2")
print("====================================")
print("1. Install library (Option A: pip, Option B: build from source)")
print("2. Load RINEX file")
print("3. (Optional) Load/download SP3 ephemeris")
print("4. Run Analysis")
print("5. View plots and export results")